In [1]:
# Fungsi untuk membaca data train 
# Membuat tag_count, word_tag, tag_trans untuk menghitung transition probability dan emission probability
def read_file_init_table(fname):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
        
    with open(fname) as f:
        content = f.readlines()

    content = [x.strip() for x in content]
    content = [l.lower() for l in content]
    idx_line = 0
    is_first_word = 0
    
    while idx_line < len(content):
        prev_tag = '<start>'
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                if content_part[1] in tag_count:
                    tag_count[content_part[1]] += 1
                else:
                    tag_count[content_part[1]] = 1
                    
                current_word_tag = content_part[0] + '|' + content_part[1]
                if current_word_tag in word_tag:
                    word_tag[current_word_tag] += 1
                else:    
                    word_tag[current_word_tag] = 1
                    
                    
                if is_first_word == 1:
                    current_tag_trans = '<start>' + '|' + content_part[1]
                    is_first_word = 0
                else:
                    current_tag_trans = prev_tag + '|' + content_part[1]
                    
                if current_tag_trans in tag_trans:
                    tag_trans[current_tag_trans] += 1
                else:
                    tag_trans[current_tag_trans] = 1                    
                prev_tag = content_part[1]                   
            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        prev_tag = '<end>'
        if prev_tag in tag_count:
            tag_count[prev_tag] += 1
        else:
            tag_count[prev_tag] = 1
        current_tag_trans = content_part[1] + '|' + prev_tag
        if current_tag_trans in tag_trans:
            tag_trans[current_tag_trans] += 1
        else:
            tag_trans[current_tag_trans] = 1                     
            
        idx_line = idx_line+1       
    return tag_count, word_tag, tag_trans

In [2]:
# Fungsi untuk membaca datatest dan menjadikan array yang berisi kata-kata yang sudah di-split di satu kalimat 
# beserta kumpulan tag dari kata-kata tersebut, kemudian diubah ke lower-case
def read_datatest(fname):
    sentences = []
    tags = []
    with open(fname) as f:
        content = f.readlines()
        
    content = [x.strip() for x in content]
    content = [x.lower() for x in content]
    idx_line = 0
    while idx_line < len(content):
        sen = []
        tag = []
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                sen.append(content_part[0])
                tag.append(content_part[1])
            idx_line = idx_line + 1
        sentences.append(sen)
        tags.append(tag)
        idx_line = idx_line+2        
    return sentences, tags

In [3]:
# Membaca datatrain
tag_count, word_tag, tag_trans = read_file_init_table('datatrain.txt')

# print(tag_count)
# print(word_tag)
# print(tag_trans)

In [4]:
# Membaca datatest
sentences, tags = read_datatest('datatest-ambiguity.txt')

In [5]:
# Fungsi untuk menghitung transition probability
def create_trans_prob_table(tag_trans, tag_count):
    trans_prob = {}
    for tag1 in tag_count.keys():
        for tag2 in tag_count.keys():
            trans_idx = tag1 + '|' + tag2
            if trans_idx in tag_trans:
                trans_prob[trans_idx] = tag_trans[trans_idx]/tag_count[tag1]
    return trans_prob

In [6]:
trans_prob = create_trans_prob_table(tag_trans, tag_count)

In [7]:
# Fungsi untuk menghitung emission probability
def create_emission_prob_table(word_tag, tag_count):
    emission_prob = {}
    for word_tag_entry in word_tag.keys():
        word_tag_split = word_tag_entry.split('|')
        current_word = word_tag_split[0]
        current_tag = word_tag_split[1]
        if word_tag_entry != '' and current_tag != '' :
            emission_key = current_word + '|' + current_tag
            emission_prob[emission_key] = word_tag[word_tag_entry]/tag_count[current_tag]    
    return emission_prob

In [8]:
emission_prob = create_emission_prob_table(word_tag, tag_count)

In [9]:
# Mencari key dari suatu kata di dalam tabel transition probability atau emission probability
def has_key(dictionary, string):
    return [k for k,v in dictionary.items() if (string) == k.split('|')[0]]

In [10]:
# Viterbi.....
def viterbi(trans_prob, emission_prob, sentence_words):
    # initialization
    viterbi_matrix = [{}]
    tag_sequence = [{}]
    
    # Mencari tag untuk kata pertama setelah start
    key_emis = has_key(emission_prob, sentence_words[0])
    tag_em = [k.split('|')[1] for k in key_emis]
    key_trans = has_key(trans_prob, '<start>')
    tag_tr = [k.split('|')[1] for k in key_trans]
    
    # Menghitung probabilitas setiap tag dari start
    for j in range(0,len(tag_em)):
        if tag_em[j] in tag_tr:
            prob = (emission_prob.get(key_emis[j]) * trans_prob.get('<start>'+'|'+tag_em[j]))
            viterbi_matrix[0][tag_em[j]] = prob

    # HMM untuk tag kata dari kata kedua sampai kata terakhir
    for i in range(1,len(sentence_words)):
        viterbi_matrix.append({})
        tag_sequence.append({})
        
        # Mencari tag untuk kata ke-i
        key_emis = has_key(emission_prob, sentence_words[i])
        tag_em = [k.split('|')[1] for k in key_emis]
        
        # Menghitung probabilitas tag dari tag sebelumnya dan mencari probabilitas terbesar tag
        for j in range(0,len(tag_em)):
            prob = {}
            for k in viterbi_matrix[i-1] :
                if(trans_prob.get(k+'|'+tag_em[j])) == None:
                    prob[k] = 0
                else:
                    prob[k] = viterbi_matrix[i-1][k] * trans_prob.get(k+'|'+tag_em[j]) * emission_prob.get(key_emis[j])

            viterbi_matrix[i][tag_em[j]] = max(prob.values())
            tag_sequence[i][tag_em[j]] = max(prob, key = prob.get)
            

    # Menghitung probabilitas kata terakhir ke end
    prob = {}
    viterbi_matrix.append({})
    for k in viterbi_matrix[i]:
        key_trans = [k for k,v in trans_prob.items() if k.endswith('<end>')]
        tag_tr = [k.split('|')[0] for k in key_trans]
        if k in tag_tr:
            prob[k] = viterbi_matrix[i][k] * trans_prob.get(k+'|'+'<end>')
        else:
            prob[k] = 0
    
    # Backward mencari tag kata dengan probabilitas yang terbesar
    m = 0
    tag = []
    tag.append(max(prob, key = prob.get))
    for l in range(len(tag_sequence)-1,0,-1):
        tag.append(tag_sequence[l].get(tag[m]))
        m += 1
    tag.reverse()   
    
    return tag

In [11]:
# Predict label dari setiap kata menggunakan viterbi
tag_predicted = []

for s in sentences:
    tag_predicted.append(viterbi(trans_prob, emission_prob, s))

In [12]:
# Save model ke text file
transmodel = open('transmodel-amb.txt', 'wt')
transmodel.write(str(trans_prob))
transmodel.close()

emissionmodel = open('emissionmodel-amb.txt', 'wt')
emissionmodel.write(str(emission_prob))
emissionmodel.close()

In [13]:
# Akurasi
def measure_accuraccy(tag_predicted, tags):
    accu = []
    for predict in range(len(tag_predicted)):
        same = 0
        for p in range(len(tag_predicted[predict])):
            if (tag_predicted[predict][p] == tags[predict][p]):
                same += 1
        accu.append(same / len(tag_predicted[predict]))
        # print(same)
    return (sum(accu)/len(accu))

In [14]:
accuraccy = measure_accuraccy(tag_predicted, tags)
print('Akurasi Viterbi : ', accuraccy)

Akurasi Viterbi :  0.9328256297403148


In [15]:
# Kalimat yang termasuk dari akurasi
for x in sentences:
     print('[%s]' % ', '.join(map(str, x)))

[bandar, udara, internasional, soekarno, â€“, hatta, merupakan, sebuah, bandar, udara, utama, yang, melayani, penerbangan, untuk, jakarta, ,, indonesia, ., bandar, udara, ini, diberi, nama, sesuai, dengan, nama, dwitunggal, tokoh, proklamator, kemerdekaan, indonesia, ,, soekarno, dan, mohammad, hatta, ,, yang, sekaligus, merupakan, presiden, dan, wakil, presiden, indonesia, pertama, .]
[bandar, udara, internasional, ngurah, rai, atau, disebut, juga, bandar, udara, internasional, i, gusti, ngurah, rai, adalah, bandar, udara, internasional, yang, terletak, di, sebelah, selatan, bali, ,, indonesia, ,, tepatnya, di, daerah, kelurahan, tuban, ,, kecamatan, kuta, ,, kabupaten, badung, ,, bali, ,, sekitar, 13, km, dari, denpasar, .]
[jenderal, ahmad, yani, adalah, salah, satu, tokoh, pahlawan, revolusi, yang, merupakan, komandan, tni, ad, yang, berhasil, merebut, kembali, kota, padang, dan, bukittinggi, pada, 1958, ., jalan, jenderal, ahmad, yani, bisa, kamu, temui, di, kota, -, kota, besar, 